# tensorflowの基本

http://docs.fabo.io/tensorflow/model_basic/tensorflow_hello.html

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib; print(device_lib.list_local_devices())

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17115982353608518845
]


In [2]:
# Creates a graph.
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
       allow_soft_placement=True, log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


Constant

定数

In [2]:
tf.constant(0)

<tf.Tensor 'Const:0' shape=() dtype=int32>

Variable

変数

In [3]:
tf.Variable(0)

<tf.Variable 'Variable:0' shape=() dtype=int32_ref>

Placeholder

In [4]:
tf.placeholder(tf.float32)

<tf.Tensor 'Placeholder:0' shape=<unknown> dtype=float32>

# 型・次元数・ランク・サイズ

In [5]:
x = tf.constant(np.random.randn(3, 3))

In [6]:
sess = tf.Session()

In [7]:
print(sess.run(x))

[[ 0.26271023 -0.62050682  0.04540681]
 [-0.28963652  0.97964158 -1.41882854]
 [ 0.71841375  0.47155554  0.47474094]]


In [8]:
print(x.dtype)

<dtype: 'float64'>


In [9]:
print(sess.run(tf.shape(x)))

[3 3]


In [10]:
print(sess.run(tf.rank(x)))

2


In [11]:
print(sess.run(tf.size(x)))

9


# 演算

In [12]:
# 2x2のプレースホルダー
x = tf.placeholder(tf.float32, shape=(2, 2))
y = tf.placeholder(tf.float32, shape=(2, 2))

In [13]:
ma = np.arange(1, 5).reshape(2, 2)
mb = np.arange(4, 8).reshape(2, 2)

In [14]:
add_op = tf.add(x, y)

In [15]:
sess = tf.Session()

In [ ]:
sess.run(add_op, feed_dict={x: ma, y: mb}) == ma + mb

In [17]:
sub_op = tf.subtract(x, y)
sess.run(sub_op, feed_dict={x: ma, y: mb}) == ma - mb

array([[ True,  True],
       [ True,  True]])

In [18]:
mul_op = tf.matmul(x, y)
sess.run(mul_op, feed_dict={x: ma, y: mb}) == np.matmul(ma, mb)

array([[ True,  True],
       [ True,  True]])

In [19]:
dot_op = tf.multiply(x, y)
sess.run(dot_op, feed_dict={x: ma, y: mb}) == ma * mb

array([[ True,  True],
       [ True,  True]])

## 例

In [20]:
cnt = tf.Variable(0, name="cnt")
inc = tf.constant(1, name="inc")

add_op = tf.add(cnt, inc)
up_op = tf.assign(cnt, add_op)  # !!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print(sess.run(up_op))
    print(sess.run(up_op))
    print(sess.run(up_op))

1
2
3


In [21]:
x = tf.constant(1, name="x")
y = tf.placeholder(tf.int32, name="y")

add_op = tf.add(x, y)

with tf.Session() as sess:    
    print(sess.run(add_op, feed_dict={y: 3}))
    print(sess.run(add_op, feed_dict={y: 5}))

4
6


# L2正則化

In [22]:
tf.set_random_seed(42)
x = tf.random_normal([10])
l2_loss_op = tf.nn.l2_loss(x)

In [23]:
with tf.Session() as sess:
    print(sess.run([x, l2_loss_op]))

[array([ 2.5322015 , -0.51309204,  0.27140725, -0.47904888,  0.03570453,
        2.3489568 , -0.08404603, -0.28674406,  0.3760416 ,  0.7895334 ],
      dtype=float32), 6.6756935]


# モデル保存と読み込み

In [24]:
tf.reset_default_graph()

In [25]:
import os
from tensorflow import gfile

a = tf.placeholder(tf.int32, name="input_a")
b = tf.placeholder(tf.int32, name="input_b")
add_op = tf.add(a, b, name="add_op")

MODEL_DIR = "./tmp/data_model/"

with tf.Session() as sess:
    os.system("rm -rf " + MODEL_DIR)
    tf.train.write_graph(sess.graph_def, MODEL_DIR, "model.pb", as_text=False)
    tf.train.write_graph(sess.graph_def, MODEL_DIR, 'model.pbtxt', as_text=True)
    print(sess.run(add_op, feed_dict={a: 1, b: 2}))

3


In [26]:
!cat ./tmp/data_model/model.pb


2
input_aPlaceholder*

shape:
2

shape:*
dtype0
(
add_opAddinput_ainput_b*
T0"

In [27]:
!cat ./tmp/data_model/model.pbtxt

node {
  name: "input_a"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "input_b"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        unknown_rank: true
      }
    }
  }
}
node {
  name: "add_op"
  op: "Add"
  input: "input_a"
  input: "input_b"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 26
}


In [28]:
def show_graph():
    for op in tf.get_default_graph().get_operations():
        for output in op.outputs:
            print("op_name={}, output_name={}".format(op.name, output.name))
show_graph()

op_name=input_a, output_name=input_a:0
op_name=input_b, output_name=input_b:0
op_name=add_op, output_name=add_op:0


In [29]:
with gfile.FastGFile(MODEL_DIR + "/model.pb", "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name="")

In [30]:
show_graph()

op_name=input_a, output_name=input_a:0
op_name=input_b, output_name=input_b:0
op_name=add_op, output_name=add_op:0
op_name=input_a_1, output_name=input_a_1:0
op_name=input_b_1, output_name=input_b_1:0
op_name=add_op_1, output_name=add_op_1:0


In [31]:
with tf.Session() as sess:
    load_a = sess.graph.get_tensor_by_name("input_a:0")
    load_b = sess.graph.get_tensor_by_name("input_b:0")
    print(load_a, load_b)
    load_op = sess.graph.get_tensor_by_name("add_op:0")
    print(load_op)
    
    print(sess.run(load_op, feed_dict={load_a: 11, load_b: 99}))

(<tf.Tensor 'input_a:0' shape=<unknown> dtype=int32>, <tf.Tensor 'input_b:0' shape=<unknown> dtype=int32>)
Tensor("add_op:0", dtype=int32)
110


# グラフ操作

In [32]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

# グラフの初期化
tf.reset_default_graph()

# グラフの取得
graph = tf.get_default_graph()
# ノード一覧
print("*** STEP1 ***", graph.get_operations())
x = tf.placeholder(tf.float32, shape=[2,2], name="matrix_x")  # 2x2 Tensor
print("*** STEP2 ***", graph.get_operations())
y = tf.placeholder(tf.float32, shape=[2,2], name="matrix_y")  # 2x2 Tensor
print("*** STEP3 ***", graph.get_operations())
matmul = tf.matmul(x, y, name="matrix_mul")  # xとyの乗算
print("*** STEP4 ***", graph.get_operations())

*** STEP1 *** []
*** STEP2 *** [<tf.Operation 'matrix_x' type=Placeholder>]
*** STEP3 *** [<tf.Operation 'matrix_x' type=Placeholder>, <tf.Operation 'matrix_y' type=Placeholder>]
*** STEP4 *** [<tf.Operation 'matrix_x' type=Placeholder>, <tf.Operation 'matrix_y' type=Placeholder>, <tf.Operation 'matrix_mul' type=MatMul>]


In [33]:
# グラフの初期化
tf.reset_default_graph()

# グラフの取得
graph = tf.get_default_graph()

# ノード一覧
x = tf.placeholder(tf.float32, shape=[2,2], name="matrix_x")  # 2x2 Tensor
y = tf.placeholder(tf.float32, shape=[2,2], name="matrix_y")  # 2x2 Tensor
matmul = tf.matmul(x, y, name="matrix_mul")  # xとyの乗算

for op in graph.get_operations():
    # ノードの名前
    print("***name***", op.name)
    # ノードの型
    print("***type***", op.type)
    # ノードの情報を表すprotocol buffer
    print("***op_def***", op.op_def)

***name*** matrix_x
***type*** Placeholder
***op_def*** name: "Placeholder"
output_arg {
  name: "output"
  type_attr: "dtype"
}
attr {
  name: "dtype"
  type: "type"
}
attr {
  name: "shape"
  type: "shape"
  default_value {
    shape {
      unknown_rank: true
    }
  }
}

***name*** matrix_y
***type*** Placeholder
***op_def*** name: "Placeholder"
output_arg {
  name: "output"
  type_attr: "dtype"
}
attr {
  name: "dtype"
  type: "type"
}
attr {
  name: "shape"
  type: "shape"
  default_value {
    shape {
      unknown_rank: true
    }
  }
}

***name*** matrix_mul
***type*** MatMul
***op_def*** name: "MatMul"
input_arg {
  name: "a"
  type_attr: "T"
}
input_arg {
  name: "b"
  type_attr: "T"
}
output_arg {
  name: "product"
  type_attr: "T"
}
attr {
  name: "transpose_a"
  type: "bool"
  default_value {
    b: false
  }
}
attr {
  name: "transpose_b"
  type: "bool"
  default_value {
    b: false
  }
}
attr {
  name: "T"
  type: "type"
  allowed_values {
    list {
      type: DT_HAL

In [34]:
# グラフの初期化
tf.reset_default_graph()

# 足し算を行うグラフを構築
a = tf.constant(1, name="a")
b = tf.constant(2, name="b")
add_op = tf.add(a, b, name="add_op")

# 方法(1)
g1 = tf.get_default_graph()
print(g1)

# 方法(2)
sess = tf.Session()
g2 = sess.graph
print(g2)

In [35]:
tf.reset_default_graph()

In [36]:
tf.reset_default_graph()

# 足し算用グラフ
tasizan_graph = tf.Graph()

# 引き算用グラフ
hikizan_graph = tf.Graph()

# 引き算グラフの構築
with tasizan_graph.as_default():
    a = tf.placeholder(tf.int32, shape=[], name="a")
    b = tf.placeholder(tf.int32, shape=[], name="b")
    add_op = tf.add(a, b, name="add_op")

    # 足し算グラフの構築
with hikizan_graph.as_default():
    x = tf.placeholder(tf.int32, shape=[], name="x")
    y = tf.placeholder(tf.int32, shape=[], name="y")
    sub_op = tf.subtract(x, y, name="sub_op")

# 足し算の実行
with tf.Session(graph=tasizan_graph) as sess:
    ret = sess.run(add_op, feed_dict={a:1,b:1})
    print(ret)

# 引き算の実行
with tf.Session(graph=hikizan_graph) as sess:
    ret = sess.run(sub_op, feed_dict={x:1,y:1})
    print(ret)

2
0


In [37]:
tf.reset_default_graph()

# 名前空間の指定
with tf.name_scope("input_layer"):
    x = tf.placeholder(shape=[64,64], dtype=tf.float32, name="x")
with tf.name_scope("first_layer"):
    w1 = tf.Variable(x, name="weights")
with tf.name_scope("second_layer"):
    # ネストを深くすることが可能
    with tf.name_scope("sub_scope"):
        a = tf.Variable([1.0], name="a")
        b = tf.Variable([2.0], name="b")
    w2 = tf.Variable(w1+a+b, name="weights")
with tf.name_scope("output_layer"):
    y = tf.Variable(w2, name="y")
g = tf.get_default_graph()
# オペレーション一覧を表示する
for op in g.get_operations():
    print(op.name)
# 可視化
# tf.summary.FileWriter('graph_log', graph=g)

input_layer/x
first_layer/weights
first_layer/weights/Assign
first_layer/weights/read
second_layer/sub_scope/a/initial_value
second_layer/sub_scope/a
second_layer/sub_scope/a/Assign
second_layer/sub_scope/a/read
second_layer/sub_scope/b/initial_value
second_layer/sub_scope/b
second_layer/sub_scope/b/Assign
second_layer/sub_scope/b/read
second_layer/add
second_layer/add_1
second_layer/weights
second_layer/weights/IsVariableInitialized
second_layer/weights/cond/Switch
second_layer/weights/cond/switch_t
second_layer/weights/cond/switch_f
second_layer/weights/cond/pred_id
second_layer/weights/cond/read
second_layer/weights/cond/read/Switch
second_layer/weights/cond/Switch_1
second_layer/weights/cond/Merge
second_layer/weights/first_layer/weights/read_second_layer/weights_0
second_layer/weights/IsVariableInitialized_1
second_layer/weights/cond_1/Switch
second_layer/weights/cond_1/switch_t
second_layer/weights/cond_1/switch_f
second_layer/weights/cond_1/pred_id
second_layer/weights/cond_1/re

In [38]:
with tf.name_scope("input_layer"):
    x = tf.placeholder(shape=[64,64], dtype=tf.float32, name="x")
with tf.name_scope("first_layer"):
    w1 = tf.Variable(x, name="weights")
with tf.name_scope("second_layer"):
    with tf.name_scope("sub_scope"):
        a = tf.Variable([1.0], name="a")
        b = tf.Variable([2.0], name="b")
    w2 = tf.Variable(w1+a+b, name="weights")
with tf.name_scope("output_layer"):
    y = tf.Variable(w2, name="y")
g = tf.get_default_graph()
w = g.get_tensor_by_name("first_layer/weights:0")
print(w)
a = g.get_tensor_by_name("second_layer/sub_scope/a:0")
print(a)

Tensor("first_layer/weights:0", shape=(64, 64), dtype=float32_ref)
Tensor("second_layer/sub_scope/a:0", shape=(1,), dtype=float32_ref)


# プロトコルバッファ


Protocol buffer

```
person {
  required name: strings "Taro Tanaka"
  optional age: int32 18
  required email: strings "taro@email.com"
}
```

XML

```
<person>
    <name>Taro Tanaka</name>
    <age>18</age>
    <email>taro@email.com</email>
</person>
```

JSON

```
{
    "person": {
        "name": "Taro Tanaka",
        "age": 18,
        "email": "taro@email.com"
    }
}
```

TensorFlowにおける流れ

1. `Graph`の生成
2. `as_graph_def()`による`GraphDef`の生成

`Graph`オブジェクトは`Tensor`とオペレーションの情報を持つオブジェクトであり、`GraphDef`オブジェクトはプロトコルバッファ用ライブラリによって生成されるオブジェクト。

- テキスト
    - 拡張子.pbtxt
    - 可読性あり
    - 編集可能
- バイナリ
    - 拡張子.pb
    - テキスト形式よりサイズが小さい

In [39]:
tf.reset_default_graph()
a = tf.constant(1)
b = tf.constant(2)
c = tf.add(a, b)
graph = tf.get_default_graph()
graph_def = graph.as_graph_def()
print(graph_def)

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 1
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "Add"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 26
}



## 学習済みモデルをprotocol bufferに保存

- ノードには名前をつける
- chekcpoint形式で保存する
```
saver = tf.train.Saver()
... 学習 ...
saver.save(sess, MODEL_DIR + '/model.ckpt')
```
- tf.Variableはconstantに変換してから保存する
```
graph_util.convert_variables_to_constants()
```



### モデルを作成してckptに保存

In [40]:
import os
import tensorflow as tf

tf.reset_default_graph()

MODEL_DIR = "./tmp/data_model/"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# graph定義
# ポイント1:重要な要素には判りやすい名前を付けておく
v1 = tf.Variable(initial_value=0.0, name="this_is_my_v1")
v1_add = v1.assign_add(1) # v1=v1+1をv1_addとして用意する

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        _v1 = sess.run(v1_add) # v1=v1+1を実行する
        print("i:%d v1=%d" % (i,_v1))

    # ポイント2:graphと学習済みv1をcheckpointに保存する
    saver.save(sess, MODEL_DIR + '/model.ckpt')

i:0 v1=1
i:1 v1=2
i:2 v1=3
i:3 v1=4
i:4 v1=5
i:5 v1=6
i:6 v1=7
i:7 v1=8
i:8 v1=9
i:9 v1=10


In [41]:
!ls tmp/data_model

checkpoint			model.ckpt.index  model.pb
model.ckpt.data-00000-of-00001	model.ckpt.meta   model.pbtxt


### モデルをckptから読み込んでpdに保存

In [42]:
from tensorflow.python.framework import graph_util

# jupyter実行用にGraphを初期化する
tf.reset_default_graph()

# checkpoint保存先ディレクトリ
MODEL_DIR = "./tmp/data_model"
# 保存先のpbファイル名
FROZEN_MODEL_NAME="frozen_model.pb"
# デバイス情報を削除する
CLEAR_DEVICES=True
# pbに書き出すoperation名
OUTPUT_NODE_NAMES="this_is_my_v1"

# graphのoperationを表示する
def print_graph_operations(graph):
    # print operations
    print("----- operations in graph -----")
    for op in graph.get_operations():
        print(op.name, op.outputs)

# graph_defのnodeを表示する
def print_graph_nodes(graph_def):
    # print nodes
    print("----- nodes in graph_def -----")
    for node in graph_def.node:
        print(node)

# checkpointファイルの確認を行う
checkpoint = tf.train.get_checkpoint_state(MODEL_DIR)
if not checkpoint:
    # checkpointファイルが見つからない
    print("cannot find checkpoint.")
else:
    # checkpointファイルから最後に保存したモデルへのパスを取得する
    last_model = checkpoint.model_checkpoint_path
    print(("load {0}".format(last_model)))

    # pbファイル名を設定する
    absolute_model_dir = "/".join(last_model.split('/')[:-1])
    frozen_model = absolute_model_dir + "/" + FROZEN_MODEL_NAME

    # checkpointのmetaファイルからGraphを読み込む
    saver = tf.train.import_meta_graph(last_model + '.meta', clear_devices=CLEAR_DEVICES)

    # graph定義を取得する
    graph = tf.get_default_graph()
    graph_def = graph.as_graph_def()

    # print operations
    print_graph_operations(graph)

    # print nodes
    # print_graph_nodes(graph_def)

    with tf.Session() as sess:
        # 学習済みモデルの値を復元する
        saver.restore(sess, last_model)

        # tf.VariableをConstに変換したgraphを取得する
        output_graph_def = graph_util.convert_variables_to_constants(
            sess,
            graph_def,
            OUTPUT_NODE_NAMES.split(",")
        )

        # pbファイルに保存
        tf.train.write_graph(output_graph_def, MODEL_DIR,
                             FROZEN_MODEL_NAME, as_text=False)

load ./tmp/data_model/model.ckpt
----- operations in graph -----
this_is_my_v1/initial_value [<tf.Tensor 'this_is_my_v1/initial_value:0' shape=() dtype=float32>]
this_is_my_v1 [<tf.Tensor 'this_is_my_v1:0' shape=() dtype=float32_ref>]
this_is_my_v1/Assign [<tf.Tensor 'this_is_my_v1/Assign:0' shape=() dtype=float32_ref>]
this_is_my_v1/read [<tf.Tensor 'this_is_my_v1/read:0' shape=() dtype=float32>]
AssignAdd/value [<tf.Tensor 'AssignAdd/value:0' shape=() dtype=float32>]
AssignAdd [<tf.Tensor 'AssignAdd:0' shape=() dtype=float32_ref>]
save/Const [<tf.Tensor 'save/Const:0' shape=() dtype=string>]
save/SaveV2/tensor_names [<tf.Tensor 'save/SaveV2/tensor_names:0' shape=(1,) dtype=string>]
save/SaveV2/shape_and_slices [<tf.Tensor 'save/SaveV2/shape_and_slices:0' shape=(1,) dtype=string>]
save/SaveV2 []
save/control_dependency [<tf.Tensor 'save/control_dependency:0' shape=() dtype=string>]
save/RestoreV2/tensor_names [<tf.Tensor 'save/RestoreV2/tensor_names:0' shape=(1,) dtype=string>]
save/R

### pbファイル読み込み

In [43]:
tf.reset_default_graph()

# checkpoint保存先ディレクトリ
MODEL_DIR = "./tmp/data_model"

# 保存先のpbファイル名
FROZEN_MODEL_NAME="frozen_model.pb"

def print_graph_operations(graph):
    # print operations
    print("----- operations in graph -----")
    for op in graph.get_operations():
        print(op.name, op.outputs)

def print_graph_nodes(graph_def):
    # print nodes
    print("----- nodes in graph_def -----")
    for node in graph_def.node:
        print(node)

def load_graph(frozen_graph_filename):
    # pbファイルを読み込みgraph定義を復元する
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # pbファイルから復元したgraph_defをカレントgraph_defに設定する
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="my_prefix",
            op_dict=None,
            producer_op_list=None
        )
    return graph

graph = load_graph(MODEL_DIR+"/"+FROZEN_MODEL_NAME)
graph_def = graph.as_graph_def()

# print operations
print_graph_operations(graph)

v1 = graph.get_tensor_by_name('my_prefix/this_is_my_v1:0') # v1のoperationの出力nodeを取得する

with tf.Session(graph=graph) as sess:
    _v1 = sess.run(v1) # v1の値を取得する
    print("v1=%d" % (_v1))

Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
----- operations in graph -----
my_prefix/this_is_my_v1 [<tf.Tensor 'my_prefix/this_is_my_v1:0' shape=() dtype=float32>]
v1=10
